# Analyze repertoire stats model performance on validation set

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import Union

%matplotlib inline
import seaborn as sns
import genetools
from IPython.display import display, Markdown

In [3]:
from malid import config, helpers, logger
from malid.external import model_evaluation
from malid.trained_model_wrappers import RepertoireClassifier
from malid.datamodels import (
    combine_classification_option_names,
)

# Analyze

In [5]:
for gene_locus in config.gene_loci_used:
    for target_obs_col in config.classification_targets:
        models_base_dir = RepertoireClassifier._get_model_base_dir(
            gene_locus=gene_locus, target_obs_column=target_obs_col
        )  # should already exist

        output_base_dir = (
            config.paths.repertoire_stats_classifier_output_dir
            / gene_locus.name
            / combine_classification_option_names(target_obs_col)
        )  # might not yet exist
        output_base_dir.mkdir(parents=True, exist_ok=True)  # create if needed

        model_output_prefix = models_base_dir / "train_smaller_model"
        results_output_prefix = output_base_dir / "train_smaller_model"

        try:
            logger.info(
                f"{gene_locus}, {target_obs_col} from {model_output_prefix} to {results_output_prefix}"
            )

            ## Load and summarize
            experiment_set = model_evaluation.ExperimentSet.load_from_disk(
                output_prefix=model_output_prefix
            )

            # Remove global fold (we trained global fold model, but now get evaluation scores on cross-validation folds only)
            # TODO: make kdict support: del self.model_outputs[:, fold_id]
            for key in experiment_set.model_outputs[:, -1].keys():
                logger.debug(f"Removing {key} (global fold)")
                del experiment_set.model_outputs[key]

            experiment_set_global_performance = experiment_set.summarize()
            experiment_set_global_performance.export_all_models(
                func_generate_classification_report_fname=lambda model_name: f"{results_output_prefix}.classification_report.{model_name}.txt",
                func_generate_confusion_matrix_fname=lambda model_name: f"{results_output_prefix}.confusion_matrix.{model_name}.png",
                dpi=72,
            )
            combined_stats = (
                experiment_set_global_performance.get_model_comparison_stats(sort=True)
            )
            combined_stats.to_csv(
                f"{results_output_prefix}.compare_model_scores.tsv",
                sep="\t",
            )
            display(
                Markdown(
                    f"## {gene_locus}, {target_obs_col} from {model_output_prefix} to {results_output_prefix}"
                )
            )
            display(combined_stats)

            ## Review binary misclassifications: Binary prediction vs ground truth
            # For binary case, make new confusion matrix of actual disease label (y) vs predicted y_binary
            # (But this changes global score metrics)
            if (
                target_obs_col.value.is_target_binary_for_repertoire_composition_classifier
            ):
                # this is a binary healthy/sick classifier
                # re-summarize with different ground truth label
                experiment_set.summarize(
                    global_evaluation_column_name=target_obs_col.value.confusion_matrix_expanded_column_name
                ).export_all_models(
                    func_generate_classification_report_fname=lambda model_name: f"{results_output_prefix}.classification_report.{model_name}.binary_vs_ground_truth.txt",
                    func_generate_confusion_matrix_fname=lambda model_name: f"{results_output_prefix}.confusion_matrix.{model_name}.binary_vs_ground_truth.png",
                    confusion_matrix_pred_label="Predicted binary label",
                    dpi=72,
                )

            ## also create the “coefficient variability” plot, over all the CV folds
            for (
                model_name,
                model_global_performance,
            ) in experiment_set_global_performance.model_global_performances.items():
                # get feature importances for each fold
                feature_importances: Union[
                    pd.DataFrame, None
                ] = model_global_performance.feature_importances

                if feature_importances is not None:
                    # feature importances are available for this model
                    fig = plt.figure(figsize=(9, 9))
                    sns.boxplot(data=feature_importances.abs(), orient="h")
                    plt.title(
                        f"Feature importance (absolute value) variability: {model_name}"
                    )
                    plt.tight_layout()
                    genetools.plots.savefig(
                        fig,
                        f"{results_output_prefix}.feature_importances.{model_name}.png",
                        dpi=72,
                    )
                    plt.close(fig)

        except Exception as err:
            logger.exception(f"{gene_locus}, {target_obs_col} failed with error: {err}")

2023-01-07 04:08:58,592 - analyze_repertoire_stats_models.ipynb - INFO - GeneLocus.BCR, TargetObsColumnEnum.disease from /users/maximz/code/boyd-immune-repertoire-classification/data/data_v_20221224/repertoire_stats/BCR/disease/train_smaller_model to /users/maximz/code/boyd-immune-repertoire-classification/out/repertoire_stats/BCR/disease/train_smaller_model


## GeneLocus.BCR, TargetObsColumnEnum.disease from /users/maximz/code/boyd-immune-repertoire-classification/data/data_v_20221224/repertoire_stats/BCR/disease/train_smaller_model to /users/maximz/code/boyd-immune-repertoire-classification/out/repertoire_stats/BCR/disease/train_smaller_model

,ROC-AUC (weighted OvO) per fold,ROC-AUC (macro OvO) per fold,au-PRC (weighted OvO) per fold,au-PRC (macro OvO) per fold,Accuracy per fold,MCC per fold,Accuracy global,MCC global,sample_size,n_abstentions,sample_size including abstentions,abstention_rate,missing_classes
lasso_multiclass,0.938 +/- 0.014 (in 3 folds),0.940 +/- 0.016 (in 3 folds),0.937 +/- 0.017 (in 3 folds),0.941 +/- 0.018 (in 3 folds),0.805 +/- 0.023 (in 3 folds),0.719 +/- 0.031 (in 3 folds),0.805,0.718,323,0,323,0.0,False
rf_multiclass,0.922 +/- 0.008 (in 3 folds),0.923 +/- 0.009 (in 3 folds),0.929 +/- 0.008 (in 3 folds),0.932 +/- 0.010 (in 3 folds),0.768 +/- 0.051 (in 3 folds),0.659 +/- 0.068 (in 3 folds),0.768,0.658,323,0,323,0.0,False
linearsvm_ovr,0.918 +/- 0.010 (in 3 folds),0.918 +/- 0.014 (in 3 folds),0.916 +/- 0.014 (in 3 folds),0.920 +/- 0.016 (in 3 folds),0.783 +/- 0.005 (in 3 folds),0.685 +/- 0.011 (in 3 folds),0.783,0.683,323,0,323,0.0,False
xgboost,0.910 +/- 0.029 (in 3 folds),0.909 +/- 0.031 (in 3 folds),0.918 +/- 0.029 (in 3 folds),0.922 +/- 0.031 (in 3 folds),0.783 +/- 0.019 (in 3 folds),0.682 +/- 0.031 (in 3 folds),0.783,0.681,323,0,323,0.0,False
ridge_cv,0.907 +/- 0.019 (in 3 folds),0.900 +/- 0.024 (in 3 folds),0.907 +/- 0.022 (in 3 folds),0.906 +/- 0.027 (in 3 folds),0.575 +/- 0.202 (in 3 folds),0.238 +/- 0.412 (in 3 folds),0.576,0.358,323,0,323,0.0,False
elasticnet_cv,0.907 +/- 0.016 (in 3 folds),0.900 +/- 0.017 (in 3 folds),0.906 +/- 0.013 (in 3 folds),0.906 +/- 0.015 (in 3 folds),0.755 +/- 0.046 (in 3 folds),0.641 +/- 0.059 (in 3 folds),0.755,0.638,323,0,323,0.0,False
lasso_cv,0.896 +/- 0.019 (in 3 folds),0.889 +/- 0.019 (in 3 folds),0.892 +/- 0.016 (in 3 folds),0.892 +/- 0.018 (in 3 folds),0.762 +/- 0.072 (in 3 folds),0.650 +/- 0.100 (in 3 folds),0.762,0.646,323,0,323,0.0,False
dummy_stratified,0.509 +/- 0.030 (in 3 folds),0.515 +/- 0.030 (in 3 folds),0.512 +/- 0.015 (in 3 folds),0.515 +/- 0.016 (in 3 folds),0.341 +/- 0.052 (in 3 folds),0.015 +/- 0.067 (in 3 folds),0.341,0.014,323,0,323,0.0,False
dummy_most_frequent,0.500 +/- 0.000 (in 3 folds),0.500 +/- 0.000 (in 3 folds),0.500 +/- 0.000 (in 3 folds),0.500 +/- 0.000 (in 3 folds),0.461 +/- 0.019 (in 3 folds),0.000 +/- 0.000 (in 3 folds),0.461,0.000,323,0,323,0.0,True


2023-01-07 04:09:04,130 - analyze_repertoire_stats_models.ipynb - INFO - GeneLocus.BCR, TargetObsColumnEnum.disease_all_demographics_present from /users/maximz/code/boyd-immune-repertoire-classification/data/data_v_20221224/repertoire_stats/BCR/disease_all_demographics_present/train_smaller_model to /users/maximz/code/boyd-immune-repertoire-classification/out/repertoire_stats/BCR/disease_all_demographics_present/train_smaller_model


## GeneLocus.BCR, TargetObsColumnEnum.disease_all_demographics_present from /users/maximz/code/boyd-immune-repertoire-classification/data/data_v_20221224/repertoire_stats/BCR/disease_all_demographics_present/train_smaller_model to /users/maximz/code/boyd-immune-repertoire-classification/out/repertoire_stats/BCR/disease_all_demographics_present/train_smaller_model

...

## GeneLocus.TCR, TargetObsColumnEnum.age_group_pediatric_healthy_only from /users/maximz/code/boyd-immune-repertoire-classification/data/data_v_20221224/repertoire_stats/TCR/age_group_pediatric_healthy_only/train_smaller_model to /users/maximz/code/boyd-immune-repertoire-classification/out/repertoire_stats/TCR/age_group_pediatric_healthy_only/train_smaller_model

,ROC-AUC (weighted OvO) per fold,ROC-AUC (macro OvO) per fold,au-PRC (weighted OvO) per fold,au-PRC (macro OvO) per fold,Accuracy per fold,MCC per fold,Accuracy global,MCC global,sample_size,n_abstentions,sample_size including abstentions,abstention_rate,missing_classes
ridge_cv,0.978 +/- 0.022 (in 3 folds),0.978 +/- 0.022 (in 3 folds),0.937 +/- 0.063 (in 3 folds),0.937 +/- 0.063 (in 3 folds),0.826 +/- 0.052 (in 3 folds),0.000 +/- 0.000 (in 3 folds),0.826,0.000,115,0,115,0.0,True
linearsvm_ovr,0.971 +/- 0.030 (in 3 folds),0.971 +/- 0.030 (in 3 folds),0.925 +/- 0.072 (in 3 folds),0.925 +/- 0.072 (in 3 folds),0.948 +/- 0.026 (in 3 folds),0.838 +/- 0.061 (in 3 folds),0.948,0.827,115,0,115,0.0,False
elasticnet_cv,0.971 +/- 0.030 (in 3 folds),0.971 +/- 0.030 (in 3 folds),0.945 +/- 0.049 (in 3 folds),0.945 +/- 0.049 (in 3 folds),0.912 +/- 0.069 (in 3 folds),0.553 +/- 0.482 (in 3 folds),0.913,0.673,115,0,115,0.0,False
lasso_multiclass,0.968 +/- 0.038 (in 3 folds),0.968 +/- 0.038 (in 3 folds),0.932 +/- 0.064 (in 3 folds),0.932 +/- 0.064 (in 3 folds),0.965 +/- 0.016 (in 3 folds),0.888 +/- 0.046 (in 3 folds),0.965,0.885,115,0,115,0.0,False
rf_multiclass,0.958 +/- 0.019 (in 3 folds),0.958 +/- 0.019 (in 3 folds),0.809 +/- 0.138 (in 3 folds),0.809 +/- 0.138 (in 3 folds),0.922 +/- 0.024 (in 3 folds),0.702 +/- 0.097 (in 3 folds),0.922,0.706,115,0,115,0.0,False
lasso_cv,0.937 +/- 0.032 (in 3 folds),0.937 +/- 0.032 (in 3 folds),0.860 +/- 0.096 (in 3 folds),0.860 +/- 0.096 (in 3 folds),0.886 +/- 0.057 (in 3 folds),0.285 +/- 0.493 (in 3 folds),0.887,0.555,115,0,115,0.0,False
xgboost,0.936 +/- 0.037 (in 3 folds),0.936 +/- 0.037 (in 3 folds),0.846 +/- 0.082 (in 3 folds),0.846 +/- 0.082 (in 3 folds),0.939 +/- 0.014 (in 3 folds),0.761 +/- 0.118 (in 3 folds),0.939,0.778,115,0,115,0.0,False
dummy_most_frequent,0.500 +/- 0.000 (in 3 folds),0.500 +/- 0.000 (in 3 folds),0.174 +/- 0.052 (in 3 folds),0.174 +/- 0.052 (in 3 folds),0.826 +/- 0.052 (in 3 folds),0.000 +/- 0.000 (in 3 folds),0.826,0.000,115,0,115,0.0,True
dummy_stratified,0.425 +/- 0.106 (in 3 folds),0.425 +/- 0.106 (in 3 folds),0.171 +/- 0.046 (in 3 folds),0.171 +/- 0.046 (in 3 folds),0.633 +/- 0.115 (in 3 folds),-0.127 +/- 0.165 (in 3 folds),0.635,-0.138,115,0,115,0.0,False


2023-01-07 04:10:56,440 - analyze_repertoire_stats_models.ipynb - INFO - GeneLocus.TCR, TargetObsColumnEnum.sex_healthy_only from /users/maximz/code/boyd-immune-repertoire-classification/data/data_v_20221224/repertoire_stats/TCR/sex_healthy_only/train_smaller_model to /users/maximz/code/boyd-immune-repertoire-classification/out/repertoire_stats/TCR/sex_healthy_only/train_smaller_model


## GeneLocus.TCR, TargetObsColumnEnum.sex_healthy_only from /users/maximz/code/boyd-immune-repertoire-classification/data/data_v_20221224/repertoire_stats/TCR/sex_healthy_only/train_smaller_model to /users/maximz/code/boyd-immune-repertoire-classification/out/repertoire_stats/TCR/sex_healthy_only/train_smaller_model

,ROC-AUC (weighted OvO) per fold,ROC-AUC (macro OvO) per fold,au-PRC (weighted OvO) per fold,au-PRC (macro OvO) per fold,Accuracy per fold,MCC per fold,Accuracy global,MCC global,sample_size,n_abstentions,sample_size including abstentions,abstention_rate,missing_classes
lasso_multiclass,0.569 +/- 0.023 (in 3 folds),0.569 +/- 0.023 (in 3 folds),0.613 +/- 0.148 (in 3 folds),0.613 +/- 0.148 (in 3 folds),0.496 +/- 0.053 (in 3 folds),0.003 +/- 0.063 (in 3 folds),0.496,-0.011,115,0,115,0.0,False
rf_multiclass,0.558 +/- 0.046 (in 3 folds),0.558 +/- 0.046 (in 3 folds),0.590 +/- 0.197 (in 3 folds),0.590 +/- 0.197 (in 3 folds),0.539 +/- 0.001 (in 3 folds),0.153 +/- 0.072 (in 3 folds),0.539,0.075,115,0,115,0.0,False
linearsvm_ovr,0.552 +/- 0.023 (in 3 folds),0.552 +/- 0.023 (in 3 folds),0.607 +/- 0.158 (in 3 folds),0.607 +/- 0.158 (in 3 folds),0.496 +/- 0.054 (in 3 folds),-0.004 +/- 0.077 (in 3 folds),0.496,-0.012,115,0,115,0.0,False
lasso_cv,0.521 +/- 0.036 (in 3 folds),0.521 +/- 0.036 (in 3 folds),0.542 +/- 0.216 (in 3 folds),0.542 +/- 0.216 (in 3 folds),0.461 +/- 0.092 (in 3 folds),0.017 +/- 0.030 (in 3 folds),0.461,-0.132,115,0,115,0.0,False
ridge_cv,0.519 +/- 0.032 (in 3 folds),0.519 +/- 0.032 (in 3 folds),0.536 +/- 0.207 (in 3 folds),0.536 +/- 0.207 (in 3 folds),0.384 +/- 0.092 (in 3 folds),0.000 +/- 0.000 (in 3 folds),0.383,-0.257,115,0,115,0.0,False
xgboost,0.517 +/- 0.030 (in 3 folds),0.517 +/- 0.030 (in 3 folds),0.565 +/- 0.204 (in 3 folds),0.565 +/- 0.204 (in 3 folds),0.522 +/- 0.064 (in 3 folds),0.133 +/- 0.145 (in 3 folds),0.522,0.041,115,0,115,0.0,False
elasticnet_cv,0.502 +/- 0.062 (in 3 folds),0.502 +/- 0.062 (in 3 folds),0.566 +/- 0.184 (in 3 folds),0.566 +/- 0.184 (in 3 folds),0.461 +/- 0.092 (in 3 folds),0.017 +/- 0.030 (in 3 folds),0.461,-0.132,115,0,115,0.0,False
dummy_most_frequent,0.500 +/- 0.000 (in 3 folds),0.500 +/- 0.000 (in 3 folds),0.513 +/- 0.168 (in 3 folds),0.513 +/- 0.168 (in 3 folds),0.384 +/- 0.092 (in 3 folds),0.000 +/- 0.000 (in 3 folds),0.383,-0.257,115,0,115,0.0,False
dummy_stratified,0.497 +/- 0.052 (in 3 folds),0.497 +/- 0.052 (in 3 folds),0.513 +/- 0.152 (in 3 folds),0.513 +/- 0.152 (in 3 folds),0.522 +/- 0.016 (in 3 folds),-0.006 +/- 0.102 (in 3 folds),0.522,0.041,115,0,115,0.0,False
